In [1]:
cd ./Les_miserables/


/home/tp-home004/fbertel/Desktop/M2_AIC/NLP/Projet/Les_miserables


In [2]:
ls

miserable_sentences.txt  miserable.txt


In [3]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
import nltk
# load ascii text and covert to lowercase

full_text = []

for filename in os.listdir(os.getcwd()):
    if (filename!='sample_data' and filename!='.config' and filename !='.ipynb_checkpoints'):
        raw_text = open(filename, 'r', encoding='utf-8').read()
        full_text.append(raw_text.lower())
flat_list = (', '.join(full_text))
print(len(flat_list))
flat_list = flat_list.rstrip()

Using TensorFlow backend.


22620632


In [4]:
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
answer = flat_list.replace("é" , "e")
answer = answer.replace("à" , "a")
answer= answer.replace("è" , "e")
answer= answer.replace("ê" , "e")
answer = ''.join(filter(whitelist.__contains__, answer))


In [5]:
len(answer)

22518128

In [6]:
nltk.download('punkt')
tokens = nltk.word_tokenize(answer)

[nltk_data] Downloading package punkt to /home/tp-
[nltk_data]     home004/fbertel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
print(tokens[:100])

['en', 'm', 'charlesfranoisbienvenu', 'myriel', 'etait', 'eveque', 'de', 'digne', 'cetait', 'un', 'vieillard', 'denviron', 'soixantequinze', 'ans', 'il', 'occupait', 'le', 'siege', 'de', 'digne', 'depuis', 'quoique', 'ce', 'detail', 'ne', 'touche', 'en', 'aucune', 'maniere', 'au', 'fond', 'meme', 'de', 'ce', 'que', 'nous', 'avons', 'a', 'raconter', 'il', 'nest', 'peutetre', 'pas', 'inutile', 'ne', 'ftce', 'que', 'pour', 'etre', 'exact', 'en', 'tout', 'dindiquer', 'ici', 'les', 'bruits', 'et', 'les', 'propos', 'qui', 'avaient', 'couru', 'sur', 'son', 'compte', 'au', 'moment', 'o', 'il', 'etait', 'arrive', 'dans', 'le', 'diocese', 'vrai', 'ou', 'faux', 'ce', 'quon', 'dit', 'des', 'hommes', 'tient', 'souvent', 'autant', 'de', 'place', 'dans', 'leur', 'vie', 'et', 'surtout', 'dans', 'leur', 'destinee', 'que', 'ce', 'quils', 'font', 'm']


In [8]:
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

Total Tokens: 3874726
Unique Tokens: 69866


In [9]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
 


In [10]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'miserable_sentences.txt'
save_doc(sequences, out_filename)

Total Sequences: 3874675


In [11]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'miserable_sentences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

lines = lines[:100000]

In [12]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences

In [13]:
print(len(nltk.word_tokenize(lines[5])))

51


In [14]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)



In [15]:
print(sequences[5])
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

[149, 1, 80, 85, 10, 150, 703, 734, 75, 6, 602, 4, 601, 1, 80, 92, 165, 13, 445, 21, 428, 12, 409, 404, 19, 207, 56, 1, 13, 9, 44, 218, 5, 393, 6, 103, 83, 20, 358, 21, 357, 9, 22, 77, 353, 12, 31, 254, 145, 7, 278]


In [16]:
print(lines[0])

en m charlesfranoisbienvenu myriel etait eveque de digne cetait un vieillard denviron soixantequinze ans il occupait le siege de digne depuis quoique ce detail ne touche en aucune maniere au fond meme de ce que nous avons a raconter il nest peutetre pas inutile ne ftce que pour etre exact en


In [17]:
from keras.utils import to_categorical
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
 

In [18]:
import numpy as np
print(np.where(y[3]==1))
print(X[4][49])

(array([145]),)
145


In [19]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            626750    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 12535)             1266035   
Total params: 2,043,685
Trainable params: 2,043,685
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
model = load_model("../WEIGHT_NLP2/model_fr_V5.h5")
model.fit(X, y, batch_size=128 , epochs=10)

Epoch 1/10
100000/100000 [==============================] - 102s 1ms/step - loss: 1.2997 - acc: 0.7063
Epoch 2/10
100000/100000 [==============================] - 107s 1ms/step - loss: 1.2676 - acc: 0.7140
Epoch 3/10
100000/100000 [==============================] - 112s 1ms/step - loss: 1.2375 - acc: 0.7194
Epoch 4/10
  1024/100000 [..............................] - ETA: 1:46 - loss: 1.1949 - acc: 0.7344

KeyboardInterrupt: 

In [30]:
model.save("../WEIGHT_NLP2/model_fr_V6.h5")

model.fit(X, y, batch_size=128 , epochs=10)

# MORE COMPLEX NEURAL NETWORK

In [40]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 50, 50)            626750    
_________________________________________________________________
lstm_17 (LSTM)               (None, 50, 100)           60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_10 (Dense)             (None, 12535)             1266035   
Total params: 2,043,685
Tra

In [41]:
for i in range(10):
    model.fit(X,y,batch_size=128, epochs=10)
    model.save("../WEIGHT_NLP2/model_fr2_V"+str(i+1)+".h5")

Epoch 1/10
100000/100000 [==============================] - 112s 1ms/step - loss: 7.0159 - acc: 0.0470
Epoch 2/10
100000/100000 [==============================] - 112s 1ms/step - loss: 6.3439 - acc: 0.0574
Epoch 3/10
100000/100000 [==============================] - 110s 1ms/step - loss: 5.9352 - acc: 0.0746
Epoch 4/10
100000/100000 [==============================] - 108s 1ms/step - loss: 5.6495 - acc: 0.1020
Epoch 5/10
100000/100000 [==============================] - 113s 1ms/step - loss: 5.3604 - acc: 0.1493
Epoch 6/10
100000/100000 [==============================] - 107s 1ms/step - loss: 5.1085 - acc: 0.2001
Epoch 7/10
100000/100000 [==============================] - 160s 2ms/step - loss: 4.9036 - acc: 0.2369
Epoch 8/10
100000/100000 [==============================] - 171s 2ms/step - loss: 4.7517 - acc: 0.2581
Epoch 9/10
100000/100000 [==============================] - 122s 1ms/step - loss: 4.6267 - acc: 0.2743
Epoch 10/10
100000/100000 [==============================] - 142s 1ms/ste

KeyboardInterrupt: 

# PREDICTION

In [42]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)


seq_length = len(lines[0].split()) - 1

# load the model

model1 = load_model("../WEIGHT_NLP2/model_fr_V5.h5")
model2 = load_model("../WEIGHT_NLP2/model_fr2_V5.h5")
# load the tokenizer
tokenizer = load(open("../WEIGHT_NLP2/tokenizer_fr.pkl", 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model1, tokenizer, seq_length, seed_text,25)
print(generated)

generated = generate_seq(model2, tokenizer, seq_length, seed_text,25)
print(generated)

dune espece de vie epouvantable faite de toute la mort quil a donneeaussi limpression futelle horrible et profonde le lendemain de lexecution et beaucoup de jours encore apres leveque parut accable la serenite presque violente du moment funebre avait disparu le fantme de la justice sociale lobsedait lui qui dordinaire revenait

de toutes ses actions comme une rubrique rue lauteur de ce livre se tourna vers lhomme du bonheur de travail et une sorte dattentat du
comme il peignait quil avait evident quil ne sait pas de routes il avait une illusion et il y a un peu de fiacre et


### MODEL V5 accuracy of 0.7